In [ ]:
import pandas as pd
import json
import os

def load_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    tweets = []
    total_tweets = len(data)
    for index, item in enumerate(data):
        tweet_id = item['Tweet_ID']
        stock_ticker = item['Target_Ticker']
        sentiment = item['Sentiment']
        tweets.append({'tweet_id': tweet_id, 'stock_ticker': stock_ticker, 'sentiment': sentiment})
    return pd.DataFrame(tweets)

def get_tweet_text(row):
    tweet_id = row['tweet_id']
    try:
        # Use snscrape with the tweet ID through the command line
        tweet = os.popen(f'snscrape --jsonl --max-results 1 twitter-tweet {tweet_id}').read()
        tweet_json = json.loads(tweet)
        print(f'Successfully fetched tweet: {tweet_id} ({row.name+1}/{len(train_data)})')  # Log progress
        return tweet_json['content']
    except Exception as e:
        print(f'Error fetching tweet {tweet_id}: {str(e)}')  # Console log for errors
        return None

# Now, load the data
train_data = load_data('TweetFinSent_Train.json')
test_data = load_data('TweetFinSent_Test.json')

# And use the function to get tweet texts
print('Fetching tweets for train_data...')  # Console log
train_data['tweet_text'] = train_data.apply(get_tweet_text, axis=1)
print('Fetching tweets for test_data...')  # Console log
test_data['tweet_text'] = test_data.apply(get_tweet_text, axis=1)

# Convert train_data and test_data to CSV
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

print('Data saved to CSV.')


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
import joblib
import requests
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestClassifier

# Load the data from CSV
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

# Preprocessing: remove rows with missing tweet_text
train_data = train_data.dropna(subset=['tweet_text'])
test_data = test_data.dropna(subset=['tweet_text'])



# Vectorization and Model building with Random Forest
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('classifier', RandomForestClassifier(class_weight='balanced', random_state=42))
])

# Rest of the code remains same

# Training the model
X_train = train_data['tweet_text']
y_train = train_data['sentiment']
pipeline.fit(X_train, y_train)

# Evaluation
X_test = test_data['tweet_text']
y_test = test_data['sentiment']
y_pred = pipeline.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Save the model to disk
filename = 'finalized_model.sav'
joblib.dump(pipeline, filename)

# Deployment - Function to get Google finance news and predict sentiment
def get_google_finance_news(stock_ticker):
    url = f'https://www.google.com/search?q={stock_ticker}+stock&tbm=nws'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    news_items = soup.find_all('div', class_='st')
    model = joblib.load(filename)
    sentiments = []
    for news in news_items:
        sentiment = model.predict([news.get_text()])[0]
        sentiments.append({
            'news_text': news.get_text(),
            'predicted_sentiment': sentiment
        })
    return sentiments

# Example usage:
stock_ticker = "TSLA" # You can replace this with any stock ticker of your choice
news_sentiments = get_google_finance_news(stock_ticker)
for item in news_sentiments:
    print(item)


Model Accuracy: 0.5914634146341463
              precision    recall  f1-score   support

    NEGATIVE       0.83      0.20      0.32        97
     NEUTRAL       0.57      0.81      0.67       395
    POSITIVE       0.61      0.45      0.52       328

    accuracy                           0.59       820
   macro avg       0.67      0.48      0.50       820
weighted avg       0.62      0.59      0.57       820



In [16]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

# Load pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

# Convert to PyTorch Dataset
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_labels = [0 if label == "NEGATIVE" else 1 if label == "NEUTRAL" else 2 for label in y_train]
test_labels = [0 if label == "NEGATIVE" else 1 if label == "NEUTRAL" else 2 for label in y_test]

train_dataset = TweetDataset(train_encodings, train_labels)
test_dataset = TweetDataset(test_encodings, test_labels)

# Train the model
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

# Evaluate the model
results = trainer.evaluate()


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm